In [1]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 22.48 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.


In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [3]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [4]:
# make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [5]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Mounted at /content/gdrive


In [6]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

 backup		     nba_detection.data   obj.zip
'Colab Notebooks'    obj.data		  yolov3
 generate_train.py   obj.names		  yolov3_custom2.cfg


In [7]:
# copy the .zip file into the root directory of cloud VM
!cp /mydrive/obj.zip ../

In [8]:
# unzip the zip file and its contents should now be in /darknet/data/obj
!unzip ../obj.zip -d data/ 

Archive:  ../obj.zip
   creating: data/obj/
  inflating: data/__MACOSX/._obj     
  inflating: data/obj/63.jpg         
  inflating: data/obj/189.jpg        
  inflating: data/obj/77.jpg         
  inflating: data/obj/162.jpg        
  inflating: data/obj/176.jpg        
  inflating: data/obj/88.jpg         
  inflating: data/obj/29.txt         
  inflating: data/obj/15.txt         
  inflating: data/obj/114.txt        
  inflating: data/obj/100.txt        
  inflating: data/obj/214.jpg        
  inflating: data/obj/128.txt        
  inflating: data/obj/200.jpg        
  inflating: data/obj/201.jpg        
  inflating: data/obj/129.txt        
  inflating: data/obj/215.jpg        
  inflating: data/obj/101.txt        
  inflating: data/obj/115.txt        
  inflating: data/obj/14.txt         
  inflating: data/obj/28.txt         
  inflating: data/obj/177.jpg        
  inflating: data/obj/89.jpg         
  inflating: data/obj/163.jpg        
  inflating: data/obj/188.jpg        
  infl

In [9]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yolov3_custom2.cfg ./yolov3_custom2.cfg

In [10]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/obj.names ./data
!cp /mydrive/obj.data  ./data

In [11]:
# upload the generate_train.py script to cloud VM from Google Drive
!cp /mydrive/generate_train.py ./

In [12]:
# generate training txt
!python generate_train.py

In [13]:
# verify train.txt can be seen in our darknet/data folder
!ls data/

__MACOSX  obj  obj.data  obj.names  train.txt


In [14]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

--2022-05-07 10:16:46--  http://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pjreddie.com/media/files/darknet53.conv.74 [following]
--2022-05-07 10:16:47--  https://pjreddie.com/media/files/darknet53.conv.74
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   103MB/s    in 1.5s    

2022-05-07 10:16:48 (103 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [1]:
# train from start
!./darknet/darknet detector train data/obj.data yolov3_custom2.cfg darknet53.conv.74 -dont_show | tee /mydrive/backup/log/train.log

/bin/bash: ./darknet/darknet: No such file or directory
tee: /mydrive/backup/log/train.log: No such file or directory


In [ ]:
# train from previous trained weights
!./darknet/darknet detector train data/obj.data yolov3_custom2.cfg /mydrive/backup/yolov3_custom2_last.weights -dont_show | tee log/train.log

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.829755), count: 3, class_loss = 0.001787, iou_loss = 0.084439, total_loss = 0.086226 
 total_bbox = 194904, rewritten_bbox = 0.055925 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.849516), count: 39, class_loss = 0.078217, iou_loss = 0.532123, total_loss = 0.610340 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.761207), count: 5, class_loss = 0.105637, iou_loss = 0.156506, total_loss = 0.262143 
 total_bbox = 194948, rewritten_bbox = 0.055912 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (